## WGAN with Gradient Penalty

Youtube Resource: https://youtu.be/Hp-jWm2SzR8


In [ ]:
print("Conditional GAN Implementation from Scratch")

Wasserstein GAN Implementation from Scratch


In [2]:
# make sure to load tensorboard or reload if already loaded 
%load_ext tensorboard

In [3]:
# start the tensorboard server (default is port 6006)
%tensorboard --logdir=logs/ --reload_interval=5

In [4]:
import torch
import torch.nn as nn

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d, num_classes, img_size):
        super(Discriminator, self).__init__()
        self.img_size = img_size
        self.disc = nn.Sequential(
            # Input: N x channels_img x 64 x 64
            # Added additional channel to the original image
            nn.Conv2d(channels_img + 1, features_d, kernel_size=4, stride=2, padding=1),    # 32x32
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4,2,1),     # 16x16
            self._block(features_d * 2, features_d * 4, 4,2,1), # 8x8
            self._block(features_d*4, features_d * 8, 4,2,1),   # 4x4
            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
        )
        self.embed = nn.Embedding(num_classes, img_size * img_size)

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False,),
            nn.InstanceNorm2d(out_channels, affine=True),    # LayerNorm <-> InstanceNorm | affine=True -> so that it has learnable parameters
            nn.LeakyReLU(0.2),
        )

    def forward(self,x, labels):
        embedding = self.embed(labels).view(labels.shape[0],1,self.img_size, self.img_size)
        x = torch.cat([x,embedding], dim=1)     # N x C x img_size(H) x img_size(W) 
        return self.disc(x)
        

In [ ]:

class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g, num_classes, img_size, embed_size):
        super(Generator, self).__init__()
        self.img_size = img_size
        self.gen = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            self._block(z_dim + embed_size, features_g*16, 4,1,0),   # N x f_g*16 x 4 x 4
            self._block(features_g*16, features_g*8, 4,2,1),    # 8x8
            self._block(features_g*8, features_g*4, 4,2,1),     # 16x16
            self._block(features_g*4, features_g*2, 4,2,1),     # 32x32
            nn.ConvTranspose2d(
                features_g*2, channels_img, kernel_size=4, stride=2, padding=1,
            ),      # 64x64
            nn.Tanh(),  # [-1,1]  
        )
        self.embed = nn.Embedding(num_classes, embed_size)


    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x, labels):
        # latent vector z: N x noise_dim x 1 x 1
        embedding = self.embed(labels).unsqueeze(2).unsqueeze(3)
        x = torch.cat([x, embedding], dim=1)
        return self.gen(x)

In [7]:
# Mean=0, standatd deviaiton = 0.02
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)
            

In [8]:
def test():
    N, in_channels, H, W = 8,3,64,64
    z_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    initialize_weights(disc)
    assert disc(x).shape == (N,1,1,1)
    
    gen = Generator(z_dim, in_channels, 8)
    z = torch.randn((N, z_dim, 1, 1))
    initialize_weights(gen)
    
    assert gen(z).shape == (N, in_channels, H, W)
    print("Success")

In [9]:
test()

Success


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# from model import Discriminator, Generator, initialize_weights

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
LEARNING_RATE = 1e-4
BATCH_SIZE = 64
IMAGE_SIZE = 64
CHANNELS_IMG = 3    # make it 1 if you are training on a grayscale image

NUM_CLASSES = 10
GEN_EMBEDDING = 100

Z_DIM = 100
NUM_EPOCHS = 5      # It will take a lot of time coz there are lots of images to process
FEATURES_DISC = 64
FEATURES_GEN = 64
CRITIC_ITERATIONS = 5
LAMBDA_GP = 10

In [12]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)

In [ ]:
# dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms, download=True)
dataset = datasets.ImageFolder(root="../dcgan/celebA_dataset/", transform=transforms)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN, NUM_CLASSES, IMAGE_SIZE, GEN_EMBEDDING).to(device)
critic = Discriminator(CHANNELS_IMG, FEATURES_DISC, NUM_CLASSES, IMAGE_SIZE).to(device)

initialize_weights(gen)
initialize_weights(critic)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0,0.9))
opt_critic = optim.Adam(critic.parameters(), lr=LEARNING_RATE, betas=(0.0,0.9))


fixed_noise = torch.randn(32, Z_DIM, 1,1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")

step = 0

## Send both the networks to training modes
gen.train()
critic.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
  )
)

In [ ]:
def gradient_penalty(critic, labels, real, fake, device="cpu"):
    BATCH_SIZE, C, H, W = real.shape
    epsilon = torch.rand((BATCH_SIZE,1,1,1)).repeat(1,C,H,W).to(device)
    interpolated_images = real * epsilon + fake *(1-epsilon)

    # Calculate critic scores
    mixed_scores = critic(interpolated_images, labels)

    # Computing gradient of the mixed scores with respect to the interpolated images
    gradient = torch.autograd.grad(
        inputs = interpolated_images,
        outputs = mixed_scores,
        grad_outputs = torch.ones_like(mixed_scores),
        create_graph = True,
        retain_graph = True,
    )[0]

    gradient = gradient.view(gradient.shape[0],-1)      # flatten all the other dimensions of the the gradient
    gradient_norm = gradient.norm(2, dim=1)     # L2 normalization
    gradient_penalty = torch.mean((gradient_norm-1)**2)
    return gradient_penalty

In [ ]:
for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, labels) in enumerate(dataloader):
        real = real.to(device)
        
        curr_batch_size = real.shape[0]
        labels = label.to(device)

        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn(BATCH_SIZE, Z_DIM, 1, 1).to(device)
            fake = gen(noise, labels)
            critic_real = critic(real, labels).reshape(-1)
            critic_fake = critic(fake, labels).reshape(-1)

            # Add the gradient penalty to the loss
            gp = gradient_penalty(critic, labels, real, fake, device=device)

            loss_critic = (
                -(torch.mean(critic_real) - torch.mean(critic_fake)) +
                LAMBDA_GP*gp
                )
            critic.zero_grad()
            loss_critic.backward(retain_graph=True)
            opt_critic.step()


        ### Train Generator: min -E(critic(gen_fake))
        output = critic(fake, labels).reshape(-1)
        loss_gen = -torch.mean(output)
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        ###  Print Losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS} Batch {batch_idx}/{len(dataloader)} \n Loss D: {loss_critic:.4f}, Loss G: {loss_gen:.4f}]")

            with torch.no_grad():
                fake = gen(noise, labels)
                # Take out (upto) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)

                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
            
            step += 1


Epoch [0/5 Batch 0/3166 
 Loss D: -1.5017, Loss G: 0.7073]
Epoch [0/5 Batch 100/3166 
 Loss D: -1.5081, Loss G: 0.7082]


In [16]:
%tensorboard --logdir=logs/ --reload_interval=5

Reusing TensorBoard on port 6008 (pid 56358), started 0:00:13 ago. (Use '!kill 56358' to kill it.)